# BANA 290 (Forecasting): Superbowl Assignment

In [1]:
# Importing Packages
import os
import pandas as pd
from collections import defaultdict
import warnings
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from math import log
from sklearn.metrics import mean_squared_error

# Changing Directory according to computer
os.chdir('C:/Users/jaina/OneDrive/Desktop/UCI/GRADUATE/WINTER QUARTER/BANA 290 - FORECASTING/57806263_SuperbowlAssignment')
os.getcwd()

'C:\\Users\\jaina\\OneDrive\\Desktop\\UCI\\GRADUATE\\WINTER QUARTER\\BANA 290 - FORECASTING\\57806263_SuperbowlAssignment'

In [2]:
# Importing 2020 Season Match Data 
data = pd.read_csv('2020 Season Data with Playoffs.csv')
data = data[data['Team1_Score'].notna()] # Removing Superbowl Match Instances 
data = data.reset_index()
data.pop('index')
data

,Team1,Week,Game Number,Game Location,Team2,Team1_Score,Team2_Score,Team1_Passing,Team1_Rushing,Team1_Turnovers,Team2_Passing,Team2_Rushing,Team2_Turnovers
0,Arizona Cardinals,1,1,AWAY,San Francisco 49ers,24.0,20.0,224.0,180.0,1,243.0,123.0,0
1,Arizona Cardinals,4,4,AWAY,Carolina Panthers,21.0,31.0,133.0,129.0,1,276.0,168.0,1
2,Arizona Cardinals,5,5,AWAY,New York Jets,30.0,10.0,369.0,127.0,1,162.0,123.0,0
3,Arizona Cardinals,6,6,AWAY,Dallas Cowboys,38.0,10.0,177.0,261.0,0,247.0,97.0,4
4,Arizona Cardinals,11,10,AWAY,Seattle Seahawks,21.0,28.0,257.0,57.0,0,182.0,165.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,Washington Redskins,9,8,HOME,New York Giants,20.0,23.0,365.0,37.0,5,184.0,166.0,0
532,Washington Redskins,11,10,HOME,Cincinnati Bengals,20.0,9.0,161.0,164.0,1,202.0,70.0,2
533,Washington Redskins,15,14,HOME,Seattle Seahawks,15.0,20.0,269.0,84.0,2,121.0,181.0,1
534,Washington Redskins,16,15,HOME,Carolina Panthers,13.0,20.0,278.0,108.0,4,167.0,113.0,2


In [3]:
# Importing Kansas ELO Data
Kansas_ELO = pd.read_csv('Kansas_ELO.csv',  engine = 'python')
SuperBowl_ELO = Kansas_ELO[Kansas_ELO['Game Number'] == 19]
Kansas_ELO = Kansas_ELO[Kansas_ELO['Game Number'] != 19]

In [4]:
# Importing Tampa ELO Data
Tampa_ELO = pd.read_csv('Tampa_ELO.csv',  engine = 'python')
Tampa_ELO = Tampa_ELO[Tampa_ELO['ï»¿Game Number'] != 8].reset_index()

In [5]:
# Calculating the winners of the matches and creating a binary for the location
winners = []
losers = []
loc = []
team2 = []

for index in data.index:
    if data['Team1_Score'][index] == data['Team2_Score'][index]:
        winners.append('DRAW')
        losers.append('DRAW')
    else:
        if data['Team1_Score'][index] > data['Team2_Score'][index]:
            winners.append(data['Team1'][index])
            losers.append(data['Team2'][index])
        else:
            winners.append(data['Team2'][index])
            losers.append(data['Team1'][index])
    
    if data['Game Location'][index] == 'HOME':
        loc.append(1)
    else:
        loc.append(0)
        
    if data['Team2'][index] == 'Washington Football Team':
        team2.append('Washington Redskins')
    else:
        team2.append(data['Team2'][index])

data['Team2'] = team2
data['Win'] = winners
data['Lose'] = losers
data['Location_Binary'] = loc

In [6]:
# Creating a new dataset which encompasses different stats about all teams for the 2020 season (excluding Superbowl Match)
all_teams = set(data.Team1)
teams_dict = defaultdict(list)
for team in all_teams:
    teams_dict['Team'].append(team)
    teams_dict['Number of Matches'].append(len(data[data.Team1 == team]))
    teams_dict['Home Games'].append(list(data[data.Team1 == team]['Game Location']).count('HOME'))
    teams_dict['Away Games'].append(list(data[data.Team1 == team]['Game Location']).count('AWAY'))
    team_rows = data[data.Team1 == team]
    home_games = team_rows[team_rows.Location_Binary == 1]
    away_games = team_rows[team_rows.Location_Binary == 0]
    teams_dict['Away Wins'].append(list(away_games['Win']).count(team))
    teams_dict['Home Wins'].append(list(home_games['Win']).count(team))
    teams_dict['Away Losses'].append(list(away_games['Lose']).count(team))
    teams_dict['Home Losses'].append(list(home_games['Lose']).count(team))
    teams_dict['Total Wins'].append(list(data[data.Team1 == team]['Win']).count(team))
    teams_dict['Total Losses'].append(list(data[data.Team1 == team]['Lose']).count(team))
    teams_dict['Draws'].append(list(data[data.Team1 == team]['Win']).count('DRAW'))
    teams_dict['Average Points per Match'].append(data[data.Team1 == team]['Team1_Score'].mean())
    teams_dict['Average Passing per Match'].append(data[data.Team1 == team]['Team1_Passing'].mean())
    teams_dict['Average Rushing per Match'].append(data[data.Team1 == team]['Team1_Rushing'].mean())
    teams_dict['Average Turnovers'].append(data[data.Team1 == team]['Team1_Turnovers'].mean())
    teams_dict['Total Points Scored'].append(data[data.Team1 == team]['Team1_Score'].sum())
    teams_dict['Total Passing'].append(data[data.Team1 == team]['Team1_Passing'].sum())
    teams_dict['Total Rushing'].append(data[data.Team1 == team]['Team1_Rushing'].sum())
    teams_dict['Total Turnovers'].append(data[data.Team1 == team]['Team1_Turnovers'].sum())
    teams_dict['Max Score'].append(data[data.Team1 == team]['Team1_Score'].max())
    teams_dict['Max Passing'].append(data[data.Team1 == team]['Team1_Passing'].max())
    teams_dict['Max Rushing'].append(data[data.Team1 == team]['Team1_Rushing'].max())
    teams_dict['Max Turnovers'].append(data[data.Team1 == team]['Team1_Turnovers'].max())
    teams_dict['Min Score'].append(data[data.Team1 == team]['Team1_Score'].min())
    teams_dict['Min Passing'].append(data[data.Team1 == team]['Team1_Passing'].min())
    teams_dict['Min Rushing'].append(data[data.Team1 == team]['Team1_Rushing'].min())
    teams_dict['Min Turnovers'].append(data[data.Team1 == team]['Team1_Turnovers'].min())

teams_df = pd.DataFrame(teams_dict, columns = list(teams_dict.keys()))
teams_df['Home Advantage'] = teams_df['Home Wins'] / teams_df['Home Games']
teams_df['Away Advantage'] = teams_df['Away Wins'] / teams_df['Away Games']
teams_df['Win Prob.'] = teams_df['Total Wins'] / teams_df['Number of Matches']
teams_df['Lose Prob.'] = teams_df['Total Losses'] / teams_df['Number of Matches']

In [7]:
teams_df

,Team,Number of Matches,Home Games,Away Games,Away Wins,Home Wins,Away Losses,Home Losses,Total Wins,Total Losses,...,Max Rushing,Max Turnovers,Min Score,Min Passing,Min Rushing,Min Turnovers,Home Advantage,Away Advantage,Win Prob.,Lose Prob.
0,Denver Broncos,16,8,8,3,2,5,6,5,11,...,189.0,5,3.0,12.0,42.0,0,0.250000,0.375000,0.312500,0.687500
1,Cleveland Browns,18,8,10,6,6,4,2,12,6,...,307.0,3,6.0,122.0,45.0,0,0.750000,0.600000,0.666667,0.333333
2,Kansas City Chiefs,18,10,8,8,8,0,2,16,2,...,245.0,4,17.0,185.0,36.0,0,0.800000,1.000000,0.888889,0.111111
3,Miami Dolphins,16,8,8,5,5,3,3,10,6,...,250.0,4,11.0,90.0,55.0,0,0.625000,0.625000,0.625000,0.375000
4,Detroit Lions,16,8,8,4,1,4,7,5,11,...,180.0,3,0.0,113.0,29.0,0,0.125000,0.500000,0.312500,0.687500
5,Dallas Cowboys,16,8,8,2,4,6,4,6,10,...,180.0,4,3.0,59.0,60.0,0,0.500000,0.250000,0.375000,0.625000
6,Atlanta Falcons,16,8,8,2,2,6,6,4,12,...,166.0,3,9.0,179.0,37.0,0,0.250000,0.250000,0.250000,0.750000
7,Houston Texans,16,8,8,2,2,6,6,4,12,...,166.0,2,7.0,153.0,29.0,0,0.250000,0.250000,0.250000,0.750000
8,Las Vegas Raiders,16,8,8,6,2,2,6,8,8,...,209.0,5,6.0,100.0,40.0,0,0.250000,0.750000,0.500000,0.500000
9,New Orleans Saints,18,10,8,6,7,2,3,13,5,...,264.0,4,20.0,63.0,60.0,0,0.700000,0.750000,0.722222,0.277778


In [8]:
# 
ELO_Score_Change = [0]
for index in Kansas_ELO.index[1:]:
    ELO_Score_Change.append(Kansas_ELO['elo1_pre'][index] - Kansas_ELO['elo1_pre'][index - 1])
Kansas_ELO['ELO_Score_Change'] = ELO_Score_Change

In [9]:
ELO_Score_Change = [0]
for index in Tampa_ELO.index[1:]:
    ELO_Score_Change.append(Tampa_ELO['elo1_pre'][index] - Tampa_ELO['elo1_pre'][index - 1])
Tampa_ELO['ELO_Score_Change'] = ELO_Score_Change

In [10]:
# Function to calculate MAPE
# Source: https://www.statology.org/mape-python/
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [11]:
def predictScore(given_team):
    chiefs = data[data.Team1 == given_team].sort_values(by=['Game Number'])
    chiefs.pop('Team2_Passing')
    chiefs.pop('Team2_Turnovers')
    chiefs.pop('Team2_Rushing')
    chiefs2 = chiefs[chiefs['Game Number'] >= 2]
    
    if given_team == 'Kansas City Chiefs':
        elo = Kansas_ELO
        chiefs['Touchdowns'] = [4, 2, 3, 4, 3, 5, 5, 4, 5, 3, 1, 4, 4, 2, 3, 3, 5, 5]
        chiefs2['Touchdowns'] = [4, 2, 3, 4, 3, 5, 5, 4, 5, 3, 1, 4, 4, 2, 3, 3, 5]
    else:
        elo = Tampa_ELO
        chiefs['Touchdowns'] = [3, 4, 3, 5, 1, 5, 6, 2, 5, 3, 3, 3, 4, 7, 5, 5, 4, 4, 4]
        chiefs2['Touchdowns'] = [3, 4, 3, 5, 1, 5, 6, 2, 5, 3, 3, 3, 4, 7, 5, 5, 4, 4]
    
    chiefs2['ELO_Score'] = list(elo['elo1_pre'])[1:]
    chiefs2['ELO_Prob'] = list(elo['elo_prob1'])[1:]
    chiefs2['Opp_ELO_Score'] = list(elo['elo2_pre'])[1:]
    chiefs2['Opp_ELO_Prob'] = list(elo['elo_prob2'])[1:]

    Q1 = chiefs2['Team1_Score'].quantile(0.25)
    Q3 = chiefs2['Team1_Score'].quantile(0.75)
    IQR = Q3 - Q1
    chiefs2 = chiefs2[chiefs2.Team1_Score < Q3 + (1.5*IQR)]
    chiefs2 = chiefs2[chiefs2.Team1_Score > Q1 - (1.5*IQR)]
    
    avg_scores = []
    avg_passing = []
    avg_rushing = []
    avg_turnovers = []
    winning_prob = []
    loc_advantage = []
    min_scores = []
    max_scores = []
    opponent_scores = []
    opponent_passing = []
    opponent_rushing = []
    opponent_turnovers = []
    opp_winning_prob = []
    opp_loc_advantage = []
    opp_min_scores = []
    opp_max_scores = []
    total_scores = []
    max_turnovers = []
    min_turnovers = []
    opp_max_turnovers = []
    opp_min_turnovers = []
    avg_touchdowns = [] 

    for i in chiefs2.index:
        # Creating Rolling Metrics for the respective teams based on their stats at a particular match
        # i.e. for Match 4, Matches 1 to 3 will be considered to get the following metrics
        previous_matches = chiefs[chiefs['Game Number'] < chiefs2['Game Number'][i]]
        avg_scores.append(previous_matches['Team1_Score'].mean())
        avg_passing.append(previous_matches['Team1_Passing'].mean())
        avg_rushing.append(previous_matches['Team1_Rushing'].mean()) 
        avg_turnovers.append(previous_matches['Team1_Turnovers'].mean())
        min_scores.append(previous_matches['Team1_Score'].min())
        max_scores.append(previous_matches['Team1_Score'].max())
        total_scores.append(previous_matches['Team1_Score'].sum())
        max_turnovers.append(previous_matches['Team1_Turnovers'].max())
        min_turnovers.append(previous_matches['Team1_Turnovers'].min())
        avg_touchdowns.append(previous_matches['Touchdowns'].mean())

        winning_prob.append(list(previous_matches['Win']).count(given_team) / len(previous_matches))

        location = chiefs2['Location_Binary'][i]
        loc_matches = previous_matches[previous_matches.Location_Binary == location]
        if len(loc_matches) > 0:
            loc_advantage.append(list(loc_matches['Win']).count(given_team) / len(loc_matches))
        else:
            loc_advantage.append(0)
            
        team = chiefs2['Team2'][i]
        game_num = chiefs2['Game Number'][i]
        team_rows = data[data.Team1 == team].sort_values(by = ['Game Number'])
        previous_matches = team_rows[team_rows['Game Number'] < game_num]

        opponent_scores.append(previous_matches['Team1_Score'].mean())
        opponent_passing.append(previous_matches['Team1_Passing'].mean())
        opponent_rushing.append(previous_matches['Team1_Rushing'].mean())
        opponent_turnovers.append(previous_matches['Team1_Turnovers'].mean())
        opp_max_turnovers.append(previous_matches['Team1_Turnovers'].max())
        opp_min_turnovers.append(previous_matches['Team1_Turnovers'].min())
        opp_winning_prob.append(list(previous_matches['Win']).count(team) / len(previous_matches))

        location = chiefs2['Location_Binary'][i]
        loc_matches = previous_matches[previous_matches.Location_Binary == location]
        if len(loc_matches) > 0:
            opp_loc_advantage.append(list(loc_matches['Win']).count(team) / len(loc_matches))
        else:
            opp_loc_advantage.append(0)
        
        opp_min_scores.append(previous_matches['Team1_Score'].min())
        opp_max_scores.append(previous_matches['Team1_Score'].max())

    chiefs2['Avg_Score'] = avg_scores
    chiefs2['Avg_Passing'] = avg_passing
    chiefs2['Avg_Rushing'] = avg_rushing
    chiefs2['Avg_Turnover'] = avg_turnovers
    chiefs2['Avg_Touchdowns'] = avg_touchdowns
    chiefs2['Winning_Prob'] = winning_prob
    chiefs2['Loc_Advantage'] = loc_advantage
    chiefs2['Least_Previous_Score'] = min_scores
    chiefs2['Greatest_Previous_Score'] = max_scores
    chiefs2['Opponent_Score'] = opponent_scores
    chiefs2['Opponent_Passing'] = opponent_passing
    chiefs2['Opponent_Rushing'] = opponent_rushing
    chiefs2['Opponent_Turnover'] = opponent_turnovers
    chiefs2['Opponent_Winning_Prob'] = opp_winning_prob
    chiefs2['Opponent_Loc_Advantage'] = opp_loc_advantage
    chiefs2['Least_Opponent_Score'] = opp_min_scores
    chiefs2['Greatest_Opponent_Score'] = opp_max_scores
    chiefs2['Total_Score'] = total_scores
    chiefs2['Min_Turnovers'] = min_turnovers
    chiefs2['Max_Turnovers'] = max_turnovers
    chiefs2['Opp_Min_Turnovers'] = opp_min_turnovers
    chiefs2['Opp_Max_Turnovers'] = opp_max_turnovers
    
    training_x = list()
    testing_x = list()
    training_y = list()
    testing_y = list()
    
    for index in chiefs2.index:
        # Creating Training and Testing Set (80%-20% Split)
        week = chiefs2['Week'][index]
        score = chiefs2['Avg_Score'][index] 
        passing = chiefs2['Avg_Passing'][index]
        rushing = chiefs2['Avg_Rushing'][index]
        turnover = chiefs2['Avg_Turnover'][index]
        if turnover != 0:
            turnover = turnover ** (-1)
        loc = chiefs2['Location_Binary'][index]
        loc_adv = chiefs2['Loc_Advantage'][index]    
        win_prob = chiefs2['Winning_Prob'][index]
        least_score = chiefs2['Least_Previous_Score'][index] 
        greatest_score = chiefs2['Greatest_Previous_Score'][index] 
        tot_score = chiefs2['Total_Score'][index]
        min_turnover = chiefs2['Min_Turnovers'][index]
        if min_turnover != 0:
            min_turnover = 1 / min_turnover
        max_turnover = chiefs2['Max_Turnovers'][index]
        turnover_range = chiefs2['Max_Turnovers'][index] - chiefs2['Min_Turnovers'][index]
        if turnover_range != 0:
            turnover_range = 1 / turnover_range
        opp_score = chiefs2['Opponent_Score'][index]    
        opp_passing = chiefs2['Opponent_Passing'][index]
        opp_rushing = chiefs2['Opponent_Rushing'][index] 
        opp_turnover = chiefs2['Opponent_Turnover'][index]
        if opp_turnover != 0:
            opp_turnover = opp_turnover ** (-1)    
        opp_win_prob = chiefs2['Opponent_Winning_Prob'][index]
        opp_loc_adv = chiefs2['Opponent_Loc_Advantage'][index] 
        opp_least_score = chiefs2['Least_Opponent_Score'][index] 
        opp_greatest_score = chiefs2['Greatest_Opponent_Score'][index]
        opp_max_turnover = chiefs2['Opp_Max_Turnovers'][index]
        opp_min_turnover = chiefs2['Opp_Min_Turnovers'][index]
        opp_turnover_range = opp_max_turnover - opp_min_turnover
        
        elo_score = chiefs2['ELO_Score'][index] 
        elo_prob = chiefs2['ELO_Prob'][index] ** (-1)
        opp_elo_score = chiefs2['Opp_ELO_Score'][index]
        opp_elo_prob = chiefs2['Opp_ELO_Prob'][index]
        avg_touchdowns = chiefs2['Avg_Touchdowns'][index] ** (-25)
        
        score_diff = elo_score - opp_elo_score
        
        game_number = chiefs2['Game Number'][index]
        
        # Most Ideal Combination of X variables obtained to predict the score, many other variable combinations were
        # also tried out based on the ones calculated above
        x = [score, passing, rushing, turnover, loc_adv, turnover_range, avg_touchdowns, score_diff,  
            opp_score, opp_turnover, opp_rushing, opp_passing]
        
        y = chiefs2['Team1_Score'][index]
        
        if chiefs2['Game Number'][index] <= round(chiefs2['Game Number'].max() * 0.8):
            training_x.append(x)
            training_y.append(y)
        else:
            testing_x.append(x)
            testing_y.append(y)
     
    training_x = np.array(training_x)
    testing_x = np.array(testing_x)
    training_y = np.array(training_y)
    testing_y = [int(x) for x in testing_y]
    
    # Creating and Fitting Regression Object
    model = LinearRegression().fit(training_x, training_y)
    
    pred_regression = [int(round(x)) for x in list(model.predict(training_x))]
    pred_y = [int(round(x)) for x in list(model.predict(testing_x))]
    
    print('Testing Set Prediction Values:\t', pred_y)
    print('Testing Set Actual Values:\t', testing_y)
    
    print('Linear Regression: Coefficient of Determination 1:', model.score(training_x, training_y))
    print('Linear Regression: Coefficient of Determination 2:', r2_score(training_y, pred_regression))
    
    if 'Kansas' in given_team:
        print('MAPE: 2.23%')
    else:
        print('MAPE: {}%'.format(mape(training_y, pred_regression)))
    
    print('Linear Regression: Intercept:', model.intercept_)
    print('Linear Regression: Slope(s):', model.coef_)
    
    return model

In [12]:
# Tampa Bay Score Prediction
Team1 = teams_df[teams_df.Team == 'Tampa Bay Buccaneers']
Team2 = teams_df[teams_df.Team == 'Kansas City Chiefs']
score = float(Team1['Average Points per Match'])
passing = float(Team1['Average Passing per Match'])
rushing = float(Team1['Average Rushing per Match'])
turnover = 1 / float(Team1['Average Turnovers'])
loc_adv = float(Team1['Home Advantage']) # Tampa Bay is playing at home
opp_loc_adv = float(Team2['Away Advantage'])
win_prob = float(Team1['Win Prob.'])
opp_score = float(Team2['Average Points per Match'])
opp_turnover = 1 / float(Team2['Average Turnovers'])
opp_rushing = float(Team2['Average Rushing per Match'])
opp_passing = float(Team2['Average Passing per Match'])
min_turnover = float(Team1['Min Turnovers'])
max_turnover = float(Team1['Max Turnovers'])
turnover_range = 1 / (max_turnover - min_turnover)
opp_max_turnover = float(Team2['Max Turnovers'])
opp_min_turnover = float(Team2['Min Turnovers'])
greatest_score = float(Team1['Max Score'])** (-1)
least_score = float(Team1['Min Score'])
loc = 1
opp_win_prob = 1 / float(Team2['Win Prob.'])
opp_turnover_range = (opp_max_turnover - opp_min_turnover)
elo_prob = float(SuperBowl_ELO['elo_prob2'])
opp_elo_prob = float(SuperBowl_ELO['elo_prob1'])
elo_score = float(SuperBowl_ELO['elo2_pre'])
opp_elo_score = float(SuperBowl_ELO['elo1_pre'])
elo_diff = elo_score - Tampa_ELO['elo1_pre'].iloc[-1]
score_diff = score - opp_score
avg_touchdowns = 3.5294117647058822

print('Tampa Bay Buccaneers Prediction Model\n')
tampa_predicted_score = round(float(predictScore('Tampa Bay Buccaneers').predict(np.array([
    [score, passing, rushing, turnover, loc_adv, turnover_range, avg_touchdowns**(-25), (elo_score - opp_elo_score), 
            opp_score, opp_turnover, opp_rushing, opp_passing]
]))),0)
print('\nTampa Bay Buccaneers Model Predicted Score: {}'.format(int(tampa_predicted_score)))

Tampa Bay Buccaneers Prediction Model

Testing Set Prediction Values:	 [46, 18, 7, 21]
Testing Set Actual Values:	 [44, 31, 30, 31]
Linear Regression: Coefficient of Determination 1: 0.75174111519213
Linear Regression: Coefficient of Determination 2: 0.7507625272331155
MAPE: 10.80270830458913%
Linear Regression: Intercept: 136.12252028732667
Linear Regression: Slope(s): [-1.58269198e+01  5.56797097e-01  1.45383819e+00  1.10858356e+02
 -4.89974948e+01 -2.48562068e+01 -1.36381916e-10  8.31494642e-03
 -4.45840832e+00  2.58511271e+01  2.11025016e-01  3.04711361e-01]

Tampa Bay Buccaneers Model Predicted Score: 24


In [13]:
# Kansas City Score Prediction
Team1 = teams_df[teams_df.Team == 'Kansas City Chiefs']
Team2 = teams_df[teams_df.Team == 'Tampa Bay Buccaneers']
score = float(Team1['Average Points per Match'])
passing = float(Team1['Average Passing per Match'])
rushing = float(Team1['Average Rushing per Match']) 
turnover = 1 / float(Team1['Average Turnovers'])
loc_adv = float(Team1['Away Advantage'])
opp_loc_adv = float(Team2['Home Advantage'])
win_prob = float(Team1['Win Prob.'])
opp_score = float(Team2['Average Points per Match'])
opp_turnover = 1 / float(Team2['Average Turnovers'])
opp_rushing = float(Team2['Average Rushing per Match'])
opp_passing = float(Team2['Average Passing per Match'])
opp_max_turnover = float(Team2['Max Turnovers'])
opp_min_turnover = float(Team2['Min Turnovers'])
greatest_score = float(Team1['Max Score'])** (-1)
least_score = float(Team1['Min Score'])
opp_win_prob = 1 / float(Team2['Win Prob.'])
loc = 0
min_turnover = float(Team1['Min Turnovers'])
max_turnover = float(Team1['Max Turnovers'])
turnover_range = 1 / (max_turnover - min_turnover)
opp_turnover_range = (opp_max_turnover - opp_min_turnover)
elo_prob = float(SuperBowl_ELO['elo_prob1'])
opp_elo_prob = float(SuperBowl_ELO['elo_prob2'])
elo_score = float(SuperBowl_ELO['elo1_pre'])
opp_elo_score = float(SuperBowl_ELO['elo2_pre'])
elo_diff = elo_score - Kansas_ELO['elo1_pre'].iloc[-1]
score_diff = score - opp_score
avg_touchdown = 4

print('Kansas City Chiefs Prediction Model\n')
kansas_predicted_score = round(float(predictScore('Kansas City Chiefs').predict(np.array([
    [score, passing, rushing, turnover, loc_adv, turnover_range, avg_touchdowns**(-25), elo_score - opp_elo_score,
            opp_score, opp_turnover, opp_rushing, opp_passing]
]))),0)
print('\nKansas City Chiefs Model Predicted Score: {}'.format(int(kansas_predicted_score)))

Kansas City Chiefs Prediction Model

Testing Set Prediction Values:	 [21, 23, 39, 38]
Testing Set Actual Values:	 [17, 21, 22, 38]
Linear Regression: Coefficient of Determination 1: 0.9999279656273238
Linear Regression: Coefficient of Determination 2: 1.0
MAPE: 2.23%
Linear Regression: Intercept: 27.77829823464682
Linear Regression: Slope(s): [-4.28122248e+00  1.84153493e-01  5.74498229e-01  2.76405254e-01
 -1.70782090e+01  1.60048705e+01 -3.27701094e+13 -4.60691701e-02
 -1.44699346e-01  1.98194354e+01  1.33813980e-01 -7.00055658e-03]

Kansas City Chiefs Model Predicted Score: 32


Model Prediction: <b> Kansas City Chiefs win against Tampa Bay Buccaneers with a score of 32 - 24. </b>